In [1]:
import cv2
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from shutil import copyfile, copy
from google.colab.patches import cv2_imshow

In [2]:
!pip install -q py-feat

     |████████████████████████████████| 6.0MB 24.2MB/s 
     |████████████████████████████████| 3.3MB 46.1MB/s 
     |████████████████████████████████| 57.4MB 51kB/s 
     |████████████████████████████████| 4.9MB 58.5MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from feat import Detector
face_model = "RetinaFace"
landmark_model = "MobileNet"
au_model = "svm"
emotion_model = "resmasknet" #resmasknet,fer, svm, rf
detector = Detector(face_model = face_model, landmark_model = landmark_model, au_model = au_model, emotion_model = emotion_model)

In [8]:
dict1 = {0:'AU01',1:'AU02', 2:'AU04', 3:'AU05', 4:'AU06', 5:'AU07', 6:'AU09', 7:'AU10', 8:'AU12',
         9:'AU14',10:'AU15', 11:'AU17', 12:'AU20', 13:'AU23',14:'AU25',15:'AU26',16:'AU28',17:'AU45'}
# dict1[16]

In [4]:
ls 'drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/audataset/files'

01_002_FAUS_Isaree.csv     02_016_FAUS_Wachiraya.csv  03_31_FAUS_manaporn.csv
01_002_FAUS_Wachiraya.csv  02_14_FAUS_anongrat.csv    03_98_FAUS_Krittika.csv
01_02_FAUS_anongrat.csv    02_186_FAUS_Isaree.csv     04_40_FAUS_anongrat.csv
01_02_FAUS_Jiratchaya.csv  02_21_FAUS_manaporn.csv    04_41_FAUS_manaporn.csv
01_02_FAUS_Kittiya.csv     02_59_FAUS_Krittika.csv    05_55_FAUS_anongrat.csv
01_02_FAUS_Krittika.csv    02_68_FAUS_Monsrung.csv    05_83_FAUS_manaporn.csv
01_02_FAUS_manaporn.csv    02_72_FAUS_Kittiya.csv     06_81_FAUS_anongrat.csv
01_02_FAUS_Monsrung.csv    03_152_FAUS_Wachiraya.csv
01_02_FAUS_Punnama.csv     03_29_FAUS_anongrat.csv


In [11]:
ls 'drive/MyDrive/file2/vdo/Wachiraya/01_mintchyy.mp4'

drive/MyDrive/file2/vdo/Wachiraya/01_mintchyy.mp4


In [6]:
ls 'drive/MyDrive/file2/done/'

face2_01_KARNMAY_P1.csv          face5_03_clubfriday.xlsx
face2_01_KARNMAY.xlsx            face6_01_mintchyy_ep2_P1.csv
face2_02_Kemisara_P1.csv         face6_01_mintchyy_ep2.xlsx
face2_02_Kemisara.xlsx           face6_02_Babyjingko_P1.csv
face5_01_jaysbabyfood_P1.csv     face6_02_Babyjingko.xlsx
face5_01_jaysbabyfood_P1.gsheet  face7_01_mintchyy_QA_100_P1.csv
face5_01_jaysbabyfood.xlsx       face7_01_mintchyy_QA_100.xlsx
face5_02_gssspotted_P1.csv       face_7_2_01_thematter_P1.csv
face5_02_gssspotted.xlsx         face_7_2_01_thematter.xlsx
face5_03_clubfriday_P1.csv       old/


In [ ]:
pred_emotion= []
filepath_frame = 'drive/MyDrive/file2/done/face7_01_mintchyy_QA_100_P1.csv'
filepath_au = 'drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/audataset/files/01_002_FAUS_Wachiraya.csv'
vdopath = 'drive/MyDrive/file2/vdo/Wachiraya/01_mintchyy.mp4'

df = pd.read_csv(filepath_frame)
dfau = pd.read_csv(filepath_au)[['frame',' AU01_c', ' AU02_c', ' AU04_c', ' AU05_c', ' AU06_c', ' AU07_c',
                            ' AU09_c', ' AU10_c', ' AU12_c', ' AU14_c', ' AU15_c', ' AU17_c',' AU20_c',
                            ' AU23_c', ' AU25_c', ' AU26_c', ' AU28_c', ' AU45_c']]
                            
dim = (224, 224)
rowimg = []
label_img = []
filename = 'lab.png'
print('df', df.shape)

for i in tqdm(range(df.shape[0])):
  seqimg = []
  for j in range(df['start_frame'][i], df['end_frame'][i],3):
    cap = cv2.VideoCapture(vdopath)
    cap.set(cv2.CAP_PROP_POS_FRAMES, j)
    success, image = cap.read()
    faceimg = detector.detect_faces(image)
    try:
      pos = faceimg[0][:4]
      cropimg =image[int(pos[1]):int(pos[3]),int(pos[0]):int(pos[2])] 
      resized = cv2.resize(cropimg, dim, interpolation=cv2.INTER_CUBIC)     
      dfsm = dfau.loc[dfau['frame'] == dfau['frame'][j]]
      dfsm = dfsm.drop('frame', axis=1)
      dfsm = dfsm.values
      # print(dfsm[0])
      idxs = np.where(dfsm[0] == 1)[0]
      for idx in range(idxs.shape[0]):
        # print(idxs[idx])
        # print(dict1[idxs[idx]])
        cv2.imwrite(filename, resized)
        FAUS = dict1[idxs[idx]]
        dst = 'AU_'+str(i)+'_'+str(j)+'_'+FAUS+'.png'
        terminal_path = "drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FAUS_lab/{0}/{1}".format(FAUS, dst)
        copyfile('/content/lab.png', terminal_path)
    except:
      pass

df (152, 8)
